In [1]:
import os
TRAIN_DIR = '../input/rsna-intracranial-hemorrhage-detection/stage_1_train_images/'
#TEST_DIR = '../input/rsna-intracranial-hemorrhage-detection/stage_1_test_images/' #dont't touch test data!

In [2]:
import keras
from keras.applications.inception_v3 import InceptionV3
from keras import layers

INPUT_SHAPE = (224, 224, 3)

conv_base = InceptionV3(include_top=False, weights='imagenet', input_tensor=None, input_shape=INPUT_SHAPE, pooling=None, classes=1000)

model = keras.models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(1000, activation='relu'))
model.add(layers.Dense(250, activation='relu'))
model.add(layers.Dense(6, activation='sigmoid'))

model.compile(
    loss='categorical_crossentropy',
    optimizer=keras.optimizers.Adam(lr=0.001),
    metrics=['accuracy']
    
)
model.summary()

Using TensorFlow backend.


87916544/87910968 [==============================] - 4s 0us/step
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, 5, 5, 2048)        21802784  
_________________________________________________________________
flatten_1 (Flatten)          (None, 51200)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1000)              51201000  
_________________________________________________________________
dense_2 (Dense)              (None, 250)               250250    
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 1506      
Total params: 73,255,540
Trainable params: 73,221,108
Non-trainable params: 34,432
_________________________________________________________________


In [3]:
import cv2
from math import ceil
import numpy as np
import pydicom

def _read(path, desired_size):
    """Will be used in DataGenerator"""
    
    dcm = pydicom.dcmread(path)
    
    try:
        img = bsb_window(dcm)
    except:
        img = np.zeros(desired_size)
    
    
    img = cv2.resize(img, desired_size[:2], interpolation=cv2.INTER_LINEAR)
    
    return img

class DataGenerator(keras.utils.Sequence):

    def __init__(self, img_dir, id_list, labels, batch_size, img_size):

        self.list_IDs = id_list
        self.labels = labels
        self.batch_size = batch_size
        self.img_size = img_size
        self.img_dir = img_dir
        self.on_epoch_end()

    def __len__(self):
        return int(ceil(len(self.indices) / self.batch_size))

    def __getitem__(self, index):
        indices = self.indices[index*self.batch_size:(index+1)*self.batch_size]
        list_IDs_temp = [self.list_IDs[k] for k in indices]
        
        if self.labels is not None:
            X, Y = self.__data_generation(list_IDs_temp)
            return X, Y
        else:
            X = self.__data_generation(list_IDs_temp)
            return X
        
    def on_epoch_end(self):
        
        
        if self.labels is not None: # for training phase we undersample and shuffle
            # keep probability of any=0 and any=1
            keep_prob = self.labels.iloc[:, 0].map({0: 0.35, 1: 0.5})
            keep = (keep_prob > np.random.rand(len(keep_prob)))
            self.indices = np.arange(len(self.list_IDs))[keep]
            np.random.shuffle(self.indices)
        else:
            self.indices = np.arange(len(self.list_IDs))

    def __data_generation(self, list_IDs_temp):
        X = np.empty((self.batch_size, *self.img_size))
        
        if self.labels is not None: # training phase
            Y = np.empty((self.batch_size, 6), dtype=np.float32)
        
            for i, ID in enumerate(list_IDs_temp):
                X[i] = np.dstack(3*[_read(self.img_dir+ID+".dcm", self.img_size)[...,-1]])
                Y[i] = self.labels.loc[ID].values
        
            return X, Y
        
        else: # test phase
            for i, ID in enumerate(list_IDs_temp):
                X[i] = np.dstack(3*[_read(self.img_dir+ID+".dcm", self.img_size)[...,-1]])
            
            return X

In [4]:
import pandas as pd

#Credit: https://www.kaggle.com/akensert/inceptionv3-prev-resnet50-keras-baseline-model
def read_testset(filename="../input/rsna-intracranial-hemorrhage-detection/stage_1_sample_submission.csv"):
    df = pd.read_csv(filename)
    df["Image"] = df["ID"].str.slice(stop=12)
    df["Diagnosis"] = df["ID"].str.slice(start=13)
    
    df = df.loc[:, ["Label", "Diagnosis", "Image"]]
    df = df.set_index(['Image', 'Diagnosis']).unstack(level=-1)
    
    return df

def read_trainset(filename="../input/rsna-intracranial-hemorrhage-detection/stage_1_train.csv"):
    df = pd.read_csv(filename)
    df["Image"] = df["ID"].str.slice(stop=12)
    df["Diagnosis"] = df["ID"].str.slice(start=13)
    
    duplicates_to_remove = [
        1598538, 1598539, 1598540, 1598541, 1598542, 1598543,
        312468,  312469,  312470,  312471,  312472,  312473,
        2708700, 2708701, 2708702, 2708703, 2708704, 2708705,
        3032994, 3032995, 3032996, 3032997, 3032998, 3032999
    ]
    
    df = df.drop(index=duplicates_to_remove)
    df = df.reset_index(drop=True)
    
    df = df.loc[:, ["Label", "Diagnosis", "Image"]]
    df = df.set_index(['Image', 'Diagnosis']).unstack(level=-1)
    
    return df

In [5]:
from sklearn.model_selection import ShuffleSplit 

train_df = read_trainset()

# k-fold splitting
ss = ShuffleSplit(n_splits=10, test_size=0.1, random_state=257).split(train_df.index)
# get indeces for one split
train_idx, valid_idx = next(ss)
train_df_kfold = train_df.iloc[train_idx]
valid_df_kfold = train_df.iloc[valid_idx]

In [6]:
predicions = None
class PredictionCheckpoint(keras.callbacks.Callback):
    
    def __init__(self, valid_df, validation_dir=TRAIN_DIR, 
                 batch_size=32, input_size=INPUT_SHAPE):
        
        self.valid_df = validation_dir
        self.valid_images_dir = validation_dir
        self.batch_size = batch_size
        self.input_size = input_size
        self.validation_predictions = []
        self.accu = []
        
    def on_train_begin(self, logs={}):
        self.validation_predictions = []
        self.accu = []
        
    def on_epoch_end(self,batch, logs={}):
        #check how we're doing on validation
        validgen = DataGenerator(img_dir=TRAIN_DIR, id_list=self.valid_df.index, labels=None, batch_size=self.batch_size, img_size=self.input_size)
        global predictions
        predictions = self.model.predict_generator(validgen, verbose=2)
        valid_len = len(valid_df_kfold.to_numpy())
        acc = sum(sum(np.equal(predictions[:valid_len], valid_df_kfold.to_numpy())))/(predictions.shape[0]*predictions.shape[1]) 
        print("Tom's Metic", acc)
        self.accu.append(acc)
        self.validation_predictions.append(predictions)        
        
pred_history = PredictionCheckpoint(valid_df_kfold, valid_df_kfold, input_size=INPUT_SHAPE)

In [7]:
gen = DataGenerator(img_dir=TRAIN_DIR,
                    id_list=train_df_kfold.index,
                    labels=train_df_kfold,
                    batch_size=32,
                    img_size=INPUT_SHAPE)

In [8]:
hist = model.fit_generator(gen,
                    epochs=5,
                    verbose=True,
                    use_multiprocessing=True,
                    workers=4,
                    callbacks=[pred_history])

Epoch 1/5
7046/7046 [==============================] - 2410s 342ms/step - loss: 0.8166 - accuracy: 0.9999
Tom's Metic 0.22180878399746995
Epoch 2/5
7034/7034 [==============================] - 2379s 338ms/step - loss: 0.8265 - accuracy: 1.0000
Tom's Metic 0.22180878399746995
Epoch 3/5
4845/7048 [===================>..........] - ETA: 12:27 - loss: 0.8264 - accuracy: 1.0000Tom's Metic 0.22180878399746995
Epoch 4/5
7052/7052 [==============================] - 2402s 341ms/step - loss: 0.8257 - accuracy: 1.0000
Tom's Metic 0.22180878399746995
Epoch 5/5
7030/7030 [==============================] - 2410s 343ms/step - loss: 0.8252 - accuracy: 1.0000
Tom's Metic 0.22180878399746995
